In [1]:
%reload_ext autoreload
%autoreload 2

import cv2

import os
import sys
sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import DataManager

from collections import defaultdict

from matplotlib.path import Path
%matplotlib inline

import pandas as pd
import numpy as np

In [4]:
stack = 'MD589'

patch_dir = '/home/yuncong/CSHL_data_patches/'
table_filepath = os.path.join(patch_dir, '%(stack)s_indices_allLandmarks_allSections.h5'%{'stack':stack})
indices_allLandmarks_allSections = pd.read_hdf(table_filepath, 'indices_allLandmarks_allSections')
grid_parameters = pd.read_hdf(table_filepath, 'grid_parameters')

In [3]:
indices_allLandmarks_allSections

,143,145,147,149,151,153,155,157,159,161,...,299,300,302,304,306,308,310,312,314,316
12N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12N_surround,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5N,NaN,NaN,NaN,NaN,"[13519, 13725, 13726, 13727, 13930, 13931, 139...","[13109, 13110, 13111, 13112, 13113, 13114, 131...","[19336, 19337, 19338, 19542, 19543, 19544, 195...","[12909, 12910, 12911, 12912, 12913, 13110, 131...","[12906, 12907, 12908, 12909, 13112, 13113, 131...","[13116, 13117, 13118, 13119, 13120, 13121, 131...",...,"[12078, 12079, 12080, 12081, 12279, 12280, 122...","[12076, 12077, 12078, 12079, 12280, 12281, 122...","[12283, 12284, 12285, 12286, 12287, 12288, 122...","[12692, 12693, 12694, 12695, 12696, 12697, 126...","[12280, 12281, 12282, 12283, 12284, 12285, 122...","[11460, 11461, 11462, 11463, 11464, 11465, 114...","[11047, 11048, 11049, 11050, 11051, 11052, 110...",NaN,NaN,NaN
5N_surround,NaN,NaN,NaN,NaN,"[11870, 11871, 11872, 11873, 11874, 12074, 120...","[11254, 11255, 11256, 11257, 11258, 11259, 112...","[17481, 17482, 17483, 17685, 17686, 17687, 176...","[11054, 11055, 11056, 11057, 11058, 11256, 112...","[11050, 11051, 11052, 11053, 11054, 11055, 110...","[11260, 11261, 11262, 11263, 11264, 11265, 112...",...,"[10224, 10225, 10226, 10227, 10424, 10425, 104...","[10222, 10223, 10224, 10225, 10424, 10425, 104...","[10428, 10429, 10430, 10431, 10432, 10433, 104...","[10838, 10839, 10840, 10841, 10842, 10843, 108...","[10425, 10426, 10427, 10428, 10429, 10430, 104...","[9604, 9605, 9606, 9607, 9608, 9609, 9610, 961...","[9192, 9193, 9194, 9195, 9196, 9197, 9198, 939...",NaN,NaN,NaN
7N,NaN,NaN,NaN,NaN,NaN,"[19953, 19954, 19955, 20159, 20160, 20161, 201...",NaN,"[19338, 19339, 19340, 19542, 19543, 19544, 195...","[18926, 18927, 18928, 19131, 19132, 19133, 191...","[18720, 18721, 18722, 18723, 18926, 18927, 189...",...,"[17478, 17479, 17682, 17683, 17684, 17685, 178...","[18098, 18099, 18100, 18101, 18102, 18103, 181...","[17889, 17890, 17891, 17892, 17893, 17894, 180...","[17060, 17061, 17265, 17266, 17267, 17268, 172...","[17471, 17472, 17473, 17474, 17677, 17678, 176...","[18919, 18920, 18921, 19123, 19124, 19125, 191...","[18506, 18507, 18712, 18713, 18714, 18917, 189...","[19534, 19535, 19739, 19740, 19741, 19742, 197...","[19946, 20152, 20153, 20154, 20155, 20357, 203...","[20564, 20769, 20770, 20771, 20975, 20976, 209..."
7N_surround,NaN,NaN,NaN,NaN,NaN,"[18096, 18097, 18098, 18099, 18100, 18101, 181...",NaN,"[17280, 17482, 17483, 17484, 17485, 17486, 174...","[16867, 16868, 17070, 17071, 17072, 17073, 170...","[16866, 16867, 16868, 16869, 16870, 16871, 168...",...,"[15622, 15623, 15624, 15625, 15626, 15826, 158...","[16244, 16245, 16246, 16247, 16248, 16249, 162...","[16034, 16035, 16036, 16037, 16038, 16039, 160...","[15203, 15204, 15205, 15206, 15207, 15208, 152...","[15616, 15617, 15618, 15619, 15620, 15820, 158...","[17063, 17064, 17065, 17066, 17067, 17068, 170...","[16650, 16651, 16652, 16653, 16654, 16855, 168...","[17682, 17884, 17885, 17886, 17887, 17888, 178...","[18294, 18295, 18296, 18297, 18298, 18299, 183...","[18709, 18710, 18711, 18913, 18914, 18915, 189..."
7n,NaN,NaN,NaN,NaN,NaN,"[14385, 14386, 14387, 14591, 14592, 14593, 145...","[14590, 14591, 14592, 14593, 14796, 14797, 147...","[14590, 14591, 14592, 14795, 14796, 14797, 147...","[15001, 15002, 15205, 15206, 15207, 15208, 152...","[15411, 15412, 15413, 15414, 15415, 15615, 156...",...,"[15399, 15400, 15604, 15605, 15606, 15607, 156...","[15194, 15399, 15400, 15401, 15604, 15605, 156...","[15194, 15195, 15196, 15197, 15198, 15399, 154...","[14785, 14990, 14991, 14992, 15195, 15196, 151...","[14785, 14786, 14991, 14992, 14993, 15196, 151...","[14581, 14582, 14786, 14787, 14788, 14789, 149...","[14170, 14171, 14376, 14377, 14581, 14582, 145...","[14171, 14377, 14378, 14582, 14583, 14584, 145...","[13965, 14171, 14172, 14377, 14378, 14583, 145...","[14378, 14584, 1

In [6]:
patch_size, stride, w, h = grid_parameters.tolist()

print 'patch_size', patch_size
print 'stride', stride
print 'w, h', w, h

half_size = patch_size/2
ys, xs = np.meshgrid(np.arange(half_size, h-half_size, stride), np.arange(half_size, w-half_size, stride),
                     indexing='xy')
sample_locations = np.c_[xs.flat, ys.flat]

patch_size 224
stride 56
w, h 13408 11744


In [5]:
byLandmark_dir = '/home/yuncong/CSHL_data_patches/%(stack)s_byLandmark/'%{'stack':stack}

In [25]:
for label in list(indices_allLandmarks_allSections.index) + ['BackG']:
    output_dir = os.path.join(byLandmark_dir, label)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

In [27]:
for sec in indices_allLandmarks_allSections.keys():
    
    if sec <= 239:
        continue
    
    print sec    
    dm = DataManager(stack=stack, section=sec, load_mask=False)
    dm._load_image(['rgb-jpg'])

    for label, indices in dict(indices_allLandmarks_allSections[sec].dropna()).iteritems():
        
        # do not dump bg patches
        if label == 'bg':
            continue
        
        for i in indices:
#         for i in np.random.choice(indices, min(10, len(indices)), replace=False):
            x, y = sample_locations[i]
            patch = dm.image_rgb_jpg[y-half_size:y+half_size, x-half_size:x+half_size]
            
            output_dir = os.path.join(byLandmark_dir, label)
            patch_name = '%(stack)s_%(sec)04d_%(gridp_ind)08d.jpg' % {'stack':stack, 'sec':sec, 'gridp_ind':i}
            cv2.imwrite(os.path.join(output_dir, patch_name), patch[..., ::-1])
            
            # if is surround, additionally dump to BackG
            if label.endswith('surround'):
                output_dir = os.path.join(byLandmark_dir, 'BackG')
                cv2.imwrite(os.path.join(output_dir, patch_name), patch[..., ::-1])

241
/oasis/projects/nsf/csd395/yuncong/CSHL_data_processed/MD589_lossless_aligned_cropped_downscaled/MD589_0241_lossless_aligned_cropped_downscaled.jpg
243
/oasis/projects/nsf/csd395/yuncong/CSHL_data_processed/MD589_lossless_aligned_cropped_downscaled/MD589_0243_lossless_aligned_cropped_downscaled.jpg
245
/oasis/projects/nsf/csd395/yuncong/CSHL_data_processed/MD589_lossless_aligned_cropped_downscaled/MD589_0245_lossless_aligned_cropped_downscaled.jpg
247
/oasis/projects/nsf/csd395/yuncong/CSHL_data_processed/MD589_lossless_aligned_cropped_downscaled/MD589_0247_lossless_aligned_cropped_downscaled.jpg
249
/oasis/projects/nsf/csd395/yuncong/CSHL_data_processed/MD589_lossless_aligned_cropped_downscaled/MD589_0249_lossless_aligned_cropped_downscaled.jpg
251
/oasis/projects/nsf/csd395/yuncong/CSHL_data_processed/MD589_lossless_aligned_cropped_downscaled/MD589_0251_lossless_aligned_cropped_downscaled.jpg
253
/oasis/projects/nsf/csd395/yuncong/CSHL_data_processed/MD589_lossless_aligned_croppe

In [6]:
# Count number of patches each landmark
for l in os.listdir(byLandmark_dir):
    print l, len(os.listdir(os.path.join(byLandmark_dir, l)))

BackG 183054
7n_surround 25119
7N 10498
Gr 2936
SuVe 939
Pn 17690
LVe_surround 8078
VLL_surround 21110
VLL 5479
LVe 2784
5N_surround 18446
12N 4359
7N_surround 32885
7n 3026
Gr_surround 19123
Pn_surround 42769
SuVe_surround 5022
12N_surround 19085
5N 5960
